In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

 Load the dataset

In [2]:
df = pd.read_csv('../data/data.csv')

1. Create Aggregate Features


We will group transactions by customer and compute aggregate statistics

In [3]:
# Aggregate features
aggregate_df = df.groupby('CustomerId').agg(
    total_transaction_amount=('Amount', 'sum'),
    avg_transaction_amount=('Amount', 'mean'),
    transaction_count=('TransactionId', 'count'),
    std_transaction_amount=('Amount', 'std')
).reset_index()

print(aggregate_df.head())

        CustomerId  total_transaction_amount  avg_transaction_amount  \
0     CustomerId_1                  -10000.0           -10000.000000   
1    CustomerId_10                  -10000.0           -10000.000000   
2  CustomerId_1001                   20000.0             4000.000000   
3  CustomerId_1002                    4225.0              384.090909   
4  CustomerId_1003                   20000.0             3333.333333   

   transaction_count  std_transaction_amount  
0                  1                     NaN  
1                  1                     NaN  
2                  5             6558.963333  
3                 11              560.498966  
4                  6             6030.478146  


2. Extract Time-based Features

We will extract features like the hour, day, month, and year from the TransactionStartTime

In [4]:
# Convert 'TransactionStartTime' to datetime if it's not in that format
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extract hour, day, month, and year
df['transaction_hour'] = df['TransactionStartTime'].dt.hour
df['transaction_day'] = df['TransactionStartTime'].dt.day
df['transaction_month'] = df['TransactionStartTime'].dt.month
df['transaction_year'] = df['TransactionStartTime'].dt.year

print(df[['transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year']].head())


   transaction_hour  transaction_day  transaction_month  transaction_year
0                 2               15                 11              2018
1                 2               15                 11              2018
2                 2               15                 11              2018
3                 3               15                 11              2018
4                 3               15                 11              2018


3. Encode Categorical Variables

In [6]:
# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['CurrencyCode', 'ProviderId', 'ProductCategory', 'ChannelId'], drop_first=True)
print(df_encoded.head())


         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId  CountryCode     ProductId   Amount  Value  \
0  CustomerId_4406          256  ProductId_10   1000.0   1000   
1  CustomerId_4406          256   ProductId_6    -20.0     20   
2  CustomerId_4683          256   ProductId_1    500.0    500   
3   CustomerId_988          256  ProductId_21  20000.0  21800   
4   CustomerId_988          256   ProductId_6   -644.0    644   

       TransactionStartTime  ...  ProductCategory_financial_services  \
0 2018-11-15 02:18:49+00:00  ...                              

Label Encoding (for ordinal categories)

In [13]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding
# le = LabelEncoder()
# df['ChannelId_encoded'] = le.fit_transform(df['ChannelId'])
# print(df[['ChannelId', 'ChannelId_encoded']].head())
# 'TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProductId'

label_encoder = LabelEncoder()
label_encoded_columns = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProductId']
for col in label_encoded_columns:
    df[col] = label_encoder.fit_transform(df[col])

print(df[['ChannelId', 'ChannelId_encoded']].head())
df.head()

     ChannelId  ChannelId_encoded
0  ChannelId_3                  2
1  ChannelId_2                  1
2  ChannelId_3                  2
3  ChannelId_3                  2
4  ChannelId_2                  1


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,ChannelId_encoded
0,78150,46980,2490,3535,2584,UGX,256,ProviderId_6,1,airtime,...,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018,2
1,75821,31755,3219,2366,2584,UGX,256,ProviderId_4,19,financial_services,...,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018,1
2,39888,60272,2713,996,2806,UGX,256,ProviderId_6,0,airtime,...,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018,2
3,48738,1797,3351,974,3733,UGX,256,ProviderId_1,11,utility_bill,...,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018,2
4,41364,48941,3219,2366,3733,UGX,256,ProviderId_4,19,financial_services,...,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018,1


4. Handle Missing Values

In [15]:
# Impute missing values with the mean for numerical columns only
df.fillna(df.select_dtypes(include=[np.number]).mean(), inplace=True)



In [16]:
# Remove rows with missing values
df.dropna(inplace=True)